In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import os

import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
static_data = pd.read_csv(f'CrossSectionalData.csv')
compositional_data = pd.read_csv(f'CompositionalLogTransform.csv')
split_data = pd.read_csv('split_datesC.txt',sep='\t',header=None,names=['date','split'])
split_data['date'] = pd.to_datetime(split_data['date'])
split_data['split'] = split_data['split'].apply(lambda x: 0 if x=='train' else 1)
static_data.head()

,LSU_id,lat,lon,area,slope,len1,csl,wid1,dep1,elev,...,SSC_TD,SS_TD,SSW_TD,SWC_TD,SWS_TD,WCC_TD,WCS_TD,WSC_TD,WSS_TD,WSW_TD
0,1,41.937259,-84.348907,10050.791910,3.280631,13936.339840,0.878460,20.507366,0.821908,313.898535,...,1.13,1.33,0.54,0.42,0.38,0.03,0.83,0.07,0.23,0.65
1,2,41.894784,-84.420307,5462.630549,3.716302,16788.937500,0.810448,14.224297,0.644027,327.838195,...,1.05,3.55,0.72,0.66,0.51,0.02,0.50,0.03,0.23,1.12
2,3,41.992562,-84.404323,4376.440964,4.515371,13638.790040,0.928196,12.452666,0.589375,335.344436,...,0.94,4.15,0.17,1.14,1.13,0.02,0.81,0.04,0.69,0.52
3,4,41.987914,-84.301405,4446.056528,3.057112,21388.652340,0.156965,12.571141,0.593107,324.147756,...,0.66,0.66,0.07,0.04,0.67,0.00,0.00,0.00,0.00,0.00
4,5,41.686662,-83.479739,247.498057,1.495516,3658.932861,0.156759,2.221942,0.186798,175.887663,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [ ]:
horizontal_shifts = np.linspace(0,300,6)
def sin_func(x,period,horizontal_shift):
    return np.sin(2 * np.pi / period * (x + horizontal_shift))

In [ ]:
_ = 1
station_dynamic_data = pd.read_csv(f'stations/processed/00000{str(_).zfill(3)}.csv')
station_dynamic_data['date'] = pd.to_datetime(station_dynamic_data['Date'])
station_dynamic_data['LSU_id'] = int(_)
station_dynamic_data['split'] = split_data['split']
for horizontal_shift in horizontal_shifts:
    station_dynamic_data[f'sin_hs_{int(horizontal_shift)}'] = sin_func(station_dynamic_data['day_of_year'], 
                                                                       365, 
                                                                       horizontal_shift
                                                                      )
station_dynamic_data.head()

,Date,pcp,max_temperature,min_temperature,rh,wnd,slr,sedyld,datenum,sinT,...,day_of_month_cos,day_of_year_sin,day_of_year_cos,week_of_year_sin,week_of_year_cos,month_of_year_sin,month_of_year_cos,date,LSU_id,split
0,1992-01-01,0.000,-1.35,-5.55,0.7970,2.4,7.86240,5.980000e-12,-2922,-0.001376,...,1.000000,0.000000,1.000000,0.0,1.0,0.0,1.0,1992-01-01,1,0
1,1992-01-02,0.000,0.95,-7.15,0.8115,1.9,8.40672,5.980000e-12,-2921,0.015826,...,0.977403,0.017261,0.999851,0.0,1.0,0.0,1.0,1992-01-02,1,0
2,1992-01-03,0.000,2.85,-3.25,0.8225,3.2,3.69792,6.640000e-06,-2920,0.033023,...,0.910635,0.034516,0.999404,0.0,1.0,0.0,1.0,1992-01-03,1,0
3,1992-01-04,3.048,3.15,0.05,0.8665,2.1,2.17728,4.460000e-03,-2919,0.050210,...,0.802712,0.051761,0.998659,0.0,1.0,0.0,1.0,1992-01-04,1,0
4,1992-01-05,2.032,6.45,-1.75,0.7975,2.9,7.94016,1.950000e-03,-2918,0.067382,...,0.658511,0.068991,0.997617,0.0,1.0,0.0,1.0,1992-01-05,1,0


In [ ]:
static_keep = ['LSU_id','area','slope','len1','csl','wid1','dep1','elev','clay', 'silt', 'sand']
station_keep = ['LSU_id', 'date', 'split'] + \
    list(station_dynamic_data.columns)[1:8] + \
    [f'sin_hs_{int(horizontal_shift)}' for horizontal_shift in horizontal_shifts]
static_data_v2 = static_data[static_keep].merge(compositional_data, on='LSU_id')
station_all_data = station_dynamic_data[station_keep].merge(static_data_v2, on='LSU_id')
station_all_data.to_csv(f'cleaned/{_}_clean_data.csv', index=False)
station_all_data.head()

,LSU_id,date,split,pcp,max_temperature,min_temperature,rh,wnd,slr,sedyld,...,SSC_TD,SS_TD,SSW_TD,SWC_TD,SWS_TD,WCC_TD,WCS_TD,WSC_TD,WSS_TD,WSW_TD
0,1,1992-01-01,0,0.000,-1.35,-5.55,0.7970,2.4,7.86240,5.980000e-12,...,0.595491,0.758452,-0.142913,-0.394227,-0.494311,-3.033285,0.286944,-2.185987,-0.996403,0.04249
1,1,1992-01-02,0,0.000,0.95,-7.15,0.8115,1.9,8.40672,5.980000e-12,...,0.595491,0.758452,-0.142913,-0.394227,-0.494311,-3.033285,0.286944,-2.185987,-0.996403,0.04249
2,1,1992-01-03,0,0.000,2.85,-3.25,0.8225,3.2,3.69792,6.640000e-06,...,0.595491,0.758452,-0.142913,-0.394227,-0.494311,-3.033285,0.286944,-2.185987,-0.996403,0.04249
3,1,1992-01-04,0,3.048,3.15,0.05,0.8665,2.1,2.17728,4.460000e-03,...,0.595491,0.758452,-0.142913,-0.394227,-0.494311,-3.033285,0.286944,-2.185987,-0.996403,0.04249
4,1,1992-01-05,0,2.032,6.45,-1.75,0.7975,2.9,7.94016,1.950000e-03,...,0.595491,0.758452,-0.142913,-0.394227,-0.494311,-3.033285,0.286944,-2.185987,-0.996403,0.04249


In [ ]:
for _ in range(2,291):
    if _ % 10 == 0:
        print(f"Processing station {_}")
    station_dynamic_data = pd.read_csv(f'stations/processed/00000{str(_).zfill(3)}.csv')
    station_dynamic_data['date'] = pd.to_datetime(station_dynamic_data['Date'])
    station_dynamic_data['LSU_id'] = int(_)
    station_dynamic_data['split'] = split_data['split']
    # make sin wave data
    for horizontal_shift in horizontal_shifts:
        station_dynamic_data[f'sin_hs_{int(horizontal_shift)}'] = sin_func(station_dynamic_data['day_of_year'], 
                                                                           365, 
                                                                           horizontal_shift
                                                                          )
    # make sure that the timing between the dates is regular (daily)
    assert len(station_dynamic_data['date'].diff().unique()) <= 2, f"Station {_} has irregular date"
    station_all_data = station_dynamic_data[station_keep].merge(static_data_v2, on='LSU_id')
    station_all_data.to_csv(f'cleaned/{_}_clean_data.csv', index=False)

Processing station 10
Processing station 20
Processing station 30
Processing station 40
Processing station 50
Processing station 60
Processing station 70
Processing station 80
Processing station 90
Processing station 100
Processing station 110
Processing station 120
Processing station 130
Processing station 140
Processing station 150
Processing station 160
Processing station 170
Processing station 180
Processing station 190
Processing station 200
Processing station 210
Processing station 220
Processing station 230
Processing station 240
Processing station 250
Processing station 260
Processing station 270
Processing station 280
Processing station 290
